In [5]:
Ruedas = {
    "base": {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ", 
         "notch": "", 
         "turnover": ""
    },
    "I"  : {
         "secuencia": "EKMFLGDQVZNTOWYHXUSPAIBRCJ", 
         "notch": "Q", 
         "turnover": "R"
    },
    "II" : {
         "secuencia": "AJDKSIRUXBLHWTMCQGZNPYFVOE", 
         "notch": "E", 
         "turnover": "F"
    },
    "III": {
         "secuencia": "BDFHJLCPRTXVZNYEIWGAKMUSQO", 
         "notch": "V", 
         "turnover": "W"
    },
    "IV" : {
         "secuencia": "ESOVPZJAYQUIRHXLNFTGKDCMWB", 
         "notch": "J", 
         "turnover": "K"
    },
    "V"  : {
         "secuencia": "VZBRGITYUPSDNHLXAWMJQOFECK", 
         "notch": "Z", 
         "turnover": "A"
    },
    "VI" : {
         "secuencia": "JPGVOUMFYQBENHZRDKASXLICTW", 
         "notch": "ZM", 
         "turnover": "AN"
    },
    "VII": {
         "secuencia": "NZJHGRCXMYSWBOUFAIVLPEKQDT", 
         "notch": "ZM", 
         "turnover": "AN"
    },
    "VIII": {
         "secuencia": "FKQHTLXOCBJSPDZRAMEWNIUYGV", 
         "notch": "ZM", 
         "turnover": "AN"
    }
}

Ruedas_griegas = {
    "base" : "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "beta" : "LEYJVCNIXWPBQMDRTAKZGFUHOS",
    "gamma": "FSOKANUERHMBTIYCWLQPZXVGJD"
}

Reflectores = {
    "base": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "A"   : "EJMZALYXVBWFCRQUONTSPIKHGD",
    "B"   : "YRUHQSLDPXNGOKMIEBFZCWVJAT",
    "C"   : "FVPJIAOYEDRZXWGCTKUQSBNMHL"
}

In [12]:
class Rotor:
    def __init__(self, rueda, ringstellung: str):      
        """
        Rueda es el mapa de la parte inicial del fichero. 
        """

        # TODO: cambiar self.wiring por self.secuencia para ser consistentes

        self.wiring = rueda["secuencia"]
        self.notch = rueda["notch"]
        self.turnover = rueda["turnover"]
        indice_ring = Ruedas["base"]["secuencia"].find(ringstellung)

        # Aplicamos el ringstellung, moviendo cada letra ciertas posiciones con respecto a la base
        
        # Los str son inmutables, así que lo cambiamos temporalmente por una lista
        self.wiring = list(self.wiring)
        
        for i in range(len(self.wiring)):
            indice = Ruedas["base"]["secuencia"].find(self.wiring[i]) + indice_ring
            self.wiring[i] = Ruedas["base"]["secuencia"][indice % 26]
        

        # Desplazamos la cadena indice_ring lugares a la derecha
        self.rotar(indice_ring)
    
        # Restauramos la cadena
        cadena = ""
        for c in self.wiring:
            cadena = cadena + c
        
        # FIXME
        #self.wiring = cadena   


    def siguiente(self, indice):
        letra_a_buscar = self.wiring[indice]
        indice_siguiente = Ruedas["base"]["secuencia"].index(letra_a_buscar)
        return indice_siguiente

    def revertir(self, indice):
        letra_en_base = Ruedas["base"]["secuencia"][indice]
        indice_de_salida = self.wiring.index(letra_en_base)    
        return indice_de_salida
    
    def rotar(self, veces: int):
        # Desplazamos la cadena veces lugares a la derecha
        self.wiring = self.wiring[len(self.wiring) - veces : ] + self.wiring[ : len(self.wiring) - veces ]


In [7]:
class Reflector:
    def __init__(self, reflector: str):
        self.wiring = reflector
    
    def reflejar(self, indice):
        """
        `Indice` representa la letra de base[indice].
        """
        letra_a_buscar = self.wiring[indice]
        indice_siguiente = Reflectores["base"]["secuencia"].find(letra_a_buscar)
        return indice_siguiente

In [8]:
class Plugboard:
    def __init__(self, parametros):
        """
        El parámetro `mapping` es una lista de duplas, de la forma
        ```
        [(letra1, letra2), (letra3, letra4)]
        ```
        
        De forma que se construirá un mapa tal que letra1 -> letra2, etc...
        """
        
        self.base = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.mapping = {}

        # Nos guardamos primero un mapa de la forma A -> A, B -> B, ...
        for letra in self.base:
            self.mapping[letra] = letra

        # Y ahora lo sustituimos por lo que nos han mandado
        for dupla in parametros:
            self.mapping[dupla[0]] = dupla[1]
            self.mapping[dupla[1]] = dupla[0]
    
    def siguiente(self, letra: str):
        return self.mapping[letra]

In [9]:
class Enigma:
    def __init__(self, ruedas: list[str], ringstellungs: str, grundstellungs: str, reflector: str, steckern: list[(str, str)]):
        if len(ruedas) != len(ringstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del ringstellung")
        if len(ruedas) != len(grundstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del grundstellung")
        
        self.rotores = []
        for rotor, ring in zip(ruedas, ringstellungs):
            self.rotores.append(Rotor(rotor, ring))
        
        self.indices = []
        for letra in grundstellungs:
            self.indices.append(Ruedas["base"]["secuencia"].find(letra))
        
        for i, veces in enumerate(self.indices):
            self.rotores[i].rotar(veces)


        self.plugboard = Plugboard(steckern)
        self.reflector = Reflector(reflector)
    
    def cifrar_caracter(self, c: str):
        # Letra -> plugbooard -> rotores -> reflector -> rotores -> plugboard -> salida
        letra = self.plugboard.siguiente(c)
        
        # Aplicar giros
        self.indices[0] = self.indices[0] + 1
        self.rotores[0].rotar(1)

        if self.indices[0] == len(Ruedas["base"]["secuencia"]):
            for i in range (1, len(self.indices)):
                self.indices[i-1] = 0

                self.indices[i] = self.indices[i] + 1
                self.rotores[i].rotar(1)

                if i != len(Ruedas["base"]["secuencia"]):
                    break
        
        # A partir de aquí, necesitamos los índices, que nos serán más cómodos. Retomaremos la letra más tarde
        indice_letra = Ruedas["base"]["secuencia"].find(letra)

        for rotor in self.rotores:
            indice_letra = rotor.siguiente(indice_letra)
        
        indice_letra = self.reflector.reflejar(indice_letra)
    
        for rotor in reversed(self.rotores):
            indice_letra = rotor.revertir(indice_letra)
        
        letra = Ruedas["base"]["secuencia"][indice_letra]

        letra = self.plugboard.siguiente(letra)
        
        return letra

    def cifrar(self, mensaje: str):
        # FIXME cuidado con la configuración inicial
        
        salida = ""

        for letra in mensaje:
            if letra == " ":
                salida = salida + " "
            else:
                salida = salida + self.cifrar_caracter(letra)
        
        return salida


In [11]:
rotores = [Ruedas["V"], Ruedas["VI"], Ruedas["VIII"], Ruedas_griegas["beta"]]
ringstellungs = "EPEL"
grundstellungs = "NAEM"
clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]

enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)

mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"
# Quitar 2 primeros y 2 últimos grupos, que sirven para advertir fallos en la transmisión        
mensaje = mensaje[10:len(mensaje)-10]
print(mensaje)

nueva_enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)

mensaje_cifrado = enigma.cifrar(mensaje)

es_conmutativo(mensaje, nueva_enigma.cifrar(mensaje_cifrado)), mensaje_cifrado

LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW


IndexError: list index out of range

In [ ]:
rotores = [Ruedas_griegas["beta"], Ruedas["V"], Ruedas["VI"], Ruedas["VIII"]]
ringstellungs = "EPEL"  # 5 16 5 12
grundstellungs = "NAEM" # 14 1 5 13

rotores = rotores[::-1]
ringstellungs = ringstellungs[::-1]
grundstellungs = grundstellungs[::-1]

clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]

enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)
print(enigma.cifrar("QEOB"))


enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)
mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"

# Quitar 2 primeros y 2 últimos grupos, que sirven para advertir fallos en la transmisión        
mensaje = mensaje[10:len(mensaje)-10]
print(mensaje)

nueva_enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)

mensaje_cifrado = enigma.cifrar(mensaje)

es_conmutativo(mensaje, nueva_enigma.cifrar(mensaje_cifrado)), mensaje_cifrado

JNQU
LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW


(True,
 'YOUY WKNC DHPU EGJJ ITZP YVBO MMKE GMQV PKCS CGPS DQXD ZWJN CKFL FGWQ XEQH FUZK BGJE TVSR LTGE SMMR WESM GOMN ECJD RCOH OFMN BVSV JLQW ESAD CGXM OSMU IHDA HCEE XHXN GERD JRNP JFKV NRIR NFUK SKAY VOEM MJUU DISU BBLN SOBC EITD BTND AGQT CJWU CQKM BRYT SVJP FSMV THIJ JTKB ETXI MAWY ZSAY MLUW SSAD CCFG XSQC KCXB WQJM JLPF PBPO PKBR OELZ NKJK FIYS PEPF DDPO BPXC IFQT RZKR BLUP BCOV JCEW XNAL AEZU ERXB BENX DSYF CKUH VQOE MONP ZASO ZCJF WFJR IEKK ENFV JLIO FRAC JYGG')